In [1]:
import torch
print("PyTorch versio:", torch.__version__)
print("CUDA GPU saatavilla:", torch.cuda.is_available())
print("Käytössä oleva GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Ei GPU:ta")


PyTorch versio: 2.5.1
CUDA GPU saatavilla: True
Käytössä oleva GPU: NVIDIA GeForce RTX 4080 SUPER


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Lataa data
tf_df = pd.read_csv(r"D:\Koulu\AIDA_S24\AIDA_project\aida-projekti-local\archive\tensorflow_preprocessed.csv")

# Skaalaa "price" MinMaxScalerilla
scaler = MinMaxScaler()
tf_df['price'] = scaler.fit_transform(tf_df[['price']])

# Määritä syöte- ja kohdeominaisuudet
target = 'price'
X = tf_df.drop(columns=[target])
y = tf_df[target]

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tarkista varianssi
print("y_train varianssi:", y_train.var())
print("y_test varianssi:", y_test.var())

# Tarkista GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Käytössä oleva laite: {device}")

print("Skaalatun y_train ensimmäiset arvot:", y_train[:10])
print("Skaalatun y_test ensimmäiset arvot:", y_test[:10])

print("y_train ensimmäiset arvot:", y_train.head())
print("y_test ensimmäiset arvot:", y_test.head())


y_train varianssi: 0.03641584232893655
y_test varianssi: 0.03627156702392699
Käytössä oleva laite: cuda
Skaalatun y_train ensimmäiset arvot: 489494     0.239207
843104     0.200537
431729     0.388707
913983     0.407364
1670638    0.815823
1270923    0.219193
94258      0.485907
504499     0.231513
1344340    0.284800
1822239    0.629333
Name: price, dtype: float64
Skaalatun y_test ensimmäiset arvot: 1562052    0.764401
871512     0.455101
169190     0.292340
1123227    0.734566
616413     0.421596
1822735    0.371839
806127     0.422536
1660688    0.323779
1314222    0.254626
1684752    0.539888
Name: price, dtype: float64
y_train ensimmäiset arvot: 489494     0.239207
843104     0.200537
431729     0.388707
913983     0.407364
1670638    0.815823
Name: price, dtype: float64
y_test ensimmäiset arvot: 1562052    0.764401
871512     0.455101
169190     0.292340
1123227    0.734566
616413     0.421596
Name: price, dtype: float64


In [3]:
# Muunna data PyTorchin tensoreiksi
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).to(device)

# Luo yksinkertainen syväoppimismalli
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.output = nn.Sequential(
            nn.Linear(64, 1),
            nn.ReLU()  # Aktivointi rajoittaa ulostulon [0, ∞) asteikolle
        )

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        return self.output(x)

model = SimpleNN(X_train_tensor.shape[1]).to(device)

# Määritä tappiofunktio ja optimointialgoritmi
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Pienennetty oppimisnopeus

# Luo DataLoader koulutusta varten
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True, num_workers=4)

# Ota käyttöön mixed precision -laskenta
scaler_grad = torch.amp.GradScaler()

# Kouluta malli
epochs = 5
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        with torch.amp.autocast(device_type='cuda'):
            predictions = model(X_batch).squeeze()
            loss = criterion(predictions, y_batch)
        scaler_grad.scale(loss).backward()
        scaler_grad.step(optimizer)
        scaler_grad.update()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.7f}")
    torch.cuda.empty_cache()

# Tee ennusteita
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor).squeeze()

# Palauta ennusteet alkuperäiseen skaalaan
predictions_clipped = torch.clamp(predictions, 0, 1)  # Varmista arvojen sopivuus skaalaan [0, 1]
predictions_original_scale = scaler.inverse_transform(predictions_clipped.cpu().numpy().reshape(-1, 1))

# Palauta y_test alkuperäiseen skaalaan vertailua varten
y_test_original_scale = scaler.inverse_transform(y_test_tensor.cpu().numpy().reshape(-1, 1))

# Näytä ensimmäiset ennusteet ja todelliset arvot alkuperäisessä skaalauksessa
print("Ensimmäiset 10 ennustetta ja todelliset arvot alkuperäisessä skaalauksessa:")
print(predictions_original_scale[:10].flatten())
print(y_test_original_scale[:10].flatten())

# GPU:n suorituskyvyn tarkistus
print("GPU:n suorituskyvyn tarkistus")
print(torch.cuda.memory_summary(device=device))


Epoch 1/5, Loss: 0.0000000
Epoch 2/5, Loss: 0.0000000
Epoch 3/5, Loss: 0.0000000
Epoch 4/5, Loss: 0.0000000
Epoch 5/5, Loss: 0.0000000
Ensimmäiset 10 ennustetta ja todelliset arvot alkuperäisessä skaalauksessa:
[1206.9193  484.     1715.5754 1958.3542  484.     1102.8667  484.
 1098.7957  484.      484.    ]
[484. 484. 484. 484. 484. 484. 484. 484. 484. 484.]
GPU:n suorituskyvyn tarkistus
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 563879 KiB |   1263 MiB |  76005 MiB